[Bloom's taxonomy](https://en.wikipedia.org/wiki/Bloom%27s_taxonomy) describes a [multilabel classification](https://en.wikipedia.org/wiki/Multi-label_classification) problem, where an intended learning objective (ILO) is labeled according to some domain criterion. For our use case, we are specifically interested in the cognitive domain, where an ILO may be assigned to either of six categories:

- create
  
- understand
  
- apply
  
- analyze
  
- evaluate
  
- remember
  
These categories reflect cognitive skills a student is expected to develop when embarking on an academic trajectory. We allow students to indicate which skill(s) they would like to focus on each time they query a recommendation from our system.

Besides students' preferences, we also need to categorize courses according to their ILOs. First, we trained a multi-layer perceptron (MLP) for multilabel classification combining back-propagation, gradient descent, and Bayesian optimization for hyper-parameter search. The MLP expects a text embedding produced by the sentence transformer `all-MiniLM-L6-v2`. It is possible to use any sentence transformer, but that would require re-training the MLP. To classify an ILO, we encode it using the sentence transformer and then pass it through the MLP. For each ILO, we then obtain a vector with six values, each corresponding to one of the six cognitive skills. These values represent the independent probability that the ILO corresponds to that cognitive skill.

Given the time it takes to compute these probabilities for all courses, we pre-compute them before setting up the recommender. Currently, these additional values are stored within `catalog.json`; found under `rec_sys_uni\datasets\data\planners\`. Each dictionary holds information about a course. To find the probabilities associated with each ILO of a specific course, look under the keyword `"proba"`.

To collapse the probabilities for each ILO into a single set of six probabilities per course, we employ the [binomial distribution](https://en.wikipedia.org/wiki/Binomial_distribution). So, for each course and for each cognitive skill, we compute the collapsed probability by evaluating the following formula:

![Collapse probabilities](equation.svg)

To learn more about how we trained the MLP, please check our notebook on [auto-labeling ILOs](Autolabeling_ILOs.ipynb). To learn more about how we collapsed the probabilities for each ILO, check our notebook on [aggregating ILO probabilities](Aggregating_ILO_Probs.ipynb).

**Note:** We don't recommend running the notebooks, because many things have changed since they were implemented, and not all files and directories may be in the places each notebook expects. However, the processes and methods used remain the same.

**Note:** The following information is deprecated. Our implementation of the Bloom-based recommender fell behind and should be upgraded to expect student inputs and course data using the new format.

The `BloomBased` class in the recommendation system is designed to generate course recommendations for students based on Bloom's Taxonomy. This taxonomy categorizes cognitive skills into six areas: create, understand, apply, analyze, evaluate, and remember. The system allows students to express their preference for specific cognitive skills, and courses are recommended accordingly.

#### Class Initialization
`BloomBased.__init__(self, precomputed_blooms: bool = True, top_n: int | None = 4, score_alg: str = 'sum', scaler: str = 'None')`
Initializes the `BloomBasedRecSys` with specified parameters.

##### Parameters
- `precomputed_blooms` (bool, optional): Determines if precomputed Bloom's values are used. Default is `True`. Currently, setting this to `False` is not implemented.
- `top_n` (int, optional): Limits the number of recommendations, applying weights according to Bloom's taxonomy. If `None`, all courses are considered.
- `score_alg` (str): Specifies the algorithm used for scoring. Current implementation supports 'sum'.
- `scaler` (str): Specifies the scaling technique for the scores. Options are 'MaxMin' or 'None'. Current default is 'None'.

#### Method: `recommend`
##### `BloomBased.recommend(self, course_data, student_blooms)`
Generates course recommendations based on student input and course data.

##### Parameters
- `course_data`: A list of dictionaries containing data about each course. The format is expected to follow that of `catalog.json`
- `student_blooms`: Bloom's taxonomy preferences indicated by the student. A dictionary with a key and value for each cognitive skill is expected.

##### Returns
- `blooms_output`: The same course data with updated scores.

##### Raises
- `AssertionError`: If `precomputed_blooms` is set to `False`.

**Notes**
- The current implementation requires `precomputed_blooms` to be `True`. Setting it to `False` will raise an error.
- The method reads precomputed Bloom's taxonomy values from a JSON file (`precomputed_blooms.json`) and uses them to match courses with student preferences.
- The `recommend` method does not currently implement any scoring or filtering based on `top_n` or `scaler` parameters, but they are included for potential future extensions.

**Deprecated Information**
- The current implementation of the Bloom-based recommender is outdated and does not support recent changes in student input and course data formats. An upgrade is recommended to align with new system requirements.
